<a href="https://colab.research.google.com/github/Pacozabala/CSCI199.X-workbench/blob/main/roberta_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
git clone https://github.com/Pacozabala/CSCI199.X-workbench

Cloning into 'CSCI199.X-workbench'...


### Start running from here if github repo is already cloned.

In [ ]:
# script to get updates
%%bash
cd CSCI199.X-workbench
git pull

Already up to date.


In [ ]:
# %%bash
# python -m spacy download en_core_web_md # needed for some spacy functions
# pip install transformers datasets scikit-learn

In [ ]:
import pandas as pd

df = pd.read_csv("CSCI199.X-workbench/data/MFRC_polarities.csv")
df = df.dropna(subset=['polarity'])
df.head()

,text,subreddit,bucket,annotator,annotation,confidence,polarity
0,That particular part of the debate is especial...,europe,French politics,annotator01,Purity,Confident,purity.virtue
2,TBH Marion Le Pen would be better. Closet fasc...,neoliberal,French politics,annotator02,Fairness,Somewhat Confident,fairness.virtue
4,The Le Pen brand of conservatism and classical...,europe,French politics,annotator03,Authority,Somewhat Confident,authority.virtue
7,"Hey, fuck you. Us leftists will never support ...",worldnews,French politics,annotator03,"Ingroup,Fairness",Confident,"ingroup.virtue,fairness.virtue"
8,"Hey, fuck you. Us leftists will never support ...",worldnews,French politics,annotator04,Purity,Confident,purity.virtue


In [ ]:
df = df[["text", "polarity"]].copy()
df = df.dropna(subset=["polarity"])

df["polarity"] = df["polarity"].str.split(",")
df = df.explode("polarity")
df["polarity"] = df["polarity"].str.strip()
df = df.drop_duplicates()


all_texts = df["text"].unique()
all_labels = sorted(df["polarity"].unique())

print(f"Found {len(all_labels)} labels:")
print(all_labels)


Found 10 labels:
['authority.vice', 'authority.virtue', 'fairness.vice', 'fairness.virtue', 'harm.vice', 'harm.virtue', 'ingroup.vice', 'ingroup.virtue', 'purity.vice', 'purity.virtue']


In [ ]:
multi_df = pd.DataFrame({"text": all_texts})

for label in all_labels:
  positive_texts = df[df["polarity"] == label]["text"].unique()
  multi_df[label] = multi_df["text"].isin(positive_texts).astype(int)

In [ ]:
virtue_vice_columns = [col for col in multi_df.columns if col.endswith(('.virtue', '.vice'))]

for col in virtue_vice_columns:
    total_count = multi_df[col].sum()
    print(f"Total count for {col}: {total_count}")

Total count for authority.vice: 108
Total count for authority.virtue: 2797
Total count for fairness.vice: 125
Total count for fairness.virtue: 4965
Total count for harm.vice: 542
Total count for harm.virtue: 3991
Total count for ingroup.vice: 69
Total count for ingroup.virtue: 1771
Total count for purity.vice: 116
Total count for purity.virtue: 1555


In [ ]:
%%bash
cd CSCI199.X-workbench
python scripts/prep_binary_data.py

Input file: data/MFRC_polarities.csv
Output dir: data/binary_datasets
Seed: 42
Test size (val+test): 0.2
Found 10 labels:
['authority.vice', 'authority.virtue', 'fairness.vice', 'fairness.virtue', 'harm.vice', 'harm.virtue', 'ingroup.vice', 'ingroup.virtue', 'purity.vice', 'purity.virtue']
Train size: 7630
Val size: 954
Test size: 954
Saved splits for: authority_vice
Saved splits for: authority_virtue
Saved splits for: fairness_vice
Saved splits for: fairness_virtue
Saved splits for: harm_vice
Saved splits for: harm_virtue
Saved splits for: ingroup_vice
Saved splits for: ingroup_virtue
Saved splits for: purity_vice
Saved splits for: purity_virtue
Finished.


In [ ]:
import pandas as pd
import os

directory_path = 'CSCI199.X-workbench/data/binary_datasets'

# Walk through all directories and subdirectories
for root, dirs, files in os.walk(directory_path):
    for filename in files:
        if filename.endswith('train.csv'):
            file_path = os.path.join(root, filename)
            try:
                df = pd.read_csv(file_path)
                if 'label' in df.columns:
                    label_sum = df['label'].sum()
                    print(f"Sum of 'label' column (from {root}): {label_sum}")
                else:
                    print(f"'{filename}' does not contain a 'label' column.")
            except Exception as e:
                print(f"Error processing {filename} (from {root}): {e}")

Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/harm_vice): 429
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/purity_virtue): 1233
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/ingroup_virtue): 1427
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/harm_virtue): 3181
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/fairness_vice): 93
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/ingroup_vice): 59
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/authority_vice): 81
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/fairness_virtue): 3969
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/purity_vice): 89
Sum of 'label' column (from CSCI199.X-workbench/data/binary_datasets/authority_virtue): 2252


In [ ]:
%%bash
pip install transformers datasets scikit-learn

In [ ]:
%%bash
cd CSCI199.X-workbench
python scripts/train_roberta.py

Epoch 1
Train Loss: 0.0688
Val F1: 0.0000
Epoch 2
Train Loss: 0.0610
Val F1: 0.0000


Loading weights: 100%|██████████| 197/197 [00:00<00:00, 2503.82it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]
RobertaForSequenceClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.weight      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on